In [2]:
import pandas as pd

train_df = pd.read_csv("train.csv")
val_df = pd.read_csv("val.csv")
test_df = pd.read_csv("test.csv")


In [3]:
pip install torch pandas sentence-transformers scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [5]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics import mean_squared_error
import numpy as np


In [6]:
encoder = SentenceTransformer("all-MiniLM-L6-v2")

X_train = encoder.encode(train_df["query"].tolist(), convert_to_tensor=True)
y_train = torch.tensor(train_df["carb"].values, dtype=torch.float32)

X_val = encoder.encode(val_df["query"].tolist(), convert_to_tensor=True)
y_val = torch.tensor(val_df["carb"].values, dtype=torch.float32)

X_test = encoder.encode(test_df["query"].tolist(), convert_to_tensor=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
class NutriDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx], self.y[idx]
        return self.X[idx]

train_ds = NutriDataset(X_train, y_train)
val_ds = NutriDataset(X_val, y_val)
test_ds = NutriDataset(X_test)

batch_size=16

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size)
test_loader = DataLoader(test_ds, batch_size=batch_size)


In [25]:
import torch.nn as nn

class MLPRegressor(nn.Module):
    def __init__(self, input_dim):
        super(MLPRegressor, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 1),
            nn.ReLU()
        )


    def forward(self, x):
        return self.net(x).squeeze(1)
model = MLPRegressor(X_train.shape[1]).to(device)


In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLPRegressor(X_train.shape[1]).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4,weight_decay=1e-5)

def train_epoch():
    model.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        preds = model(X_batch)
        loss = criterion(preds, y_batch)
        loss.backward()
        optimizer.step()

def evaluate(loader):
    model.eval()
    all_preds, all_targets = [], []
    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch = X_batch.to(device)
            preds = model(X_batch).cpu().numpy()
            all_preds.extend(preds)
            all_targets.extend(y_batch.numpy())
    return np.sqrt(mean_squared_error(all_targets, all_preds))

# Training Loop
for epoch in range(100):
    train_epoch()
    rmse = evaluate(val_loader)
    print(f"Epoch {epoch+1}: Validation RMSE = {rmse:.4f}")


Epoch 1: Validation RMSE = 42.4970
Epoch 2: Validation RMSE = 40.7793
Epoch 3: Validation RMSE = 38.7106
Epoch 4: Validation RMSE = 36.6939
Epoch 5: Validation RMSE = 35.8977
Epoch 6: Validation RMSE = 34.0133
Epoch 7: Validation RMSE = 33.5725
Epoch 8: Validation RMSE = 31.6951
Epoch 9: Validation RMSE = 30.0495
Epoch 10: Validation RMSE = 29.5834
Epoch 11: Validation RMSE = 29.4211
Epoch 12: Validation RMSE = 28.2071
Epoch 13: Validation RMSE = 28.3791
Epoch 14: Validation RMSE = 27.7948
Epoch 15: Validation RMSE = 27.4669
Epoch 16: Validation RMSE = 26.9996
Epoch 17: Validation RMSE = 26.3575
Epoch 18: Validation RMSE = 26.6093
Epoch 19: Validation RMSE = 26.3201
Epoch 20: Validation RMSE = 27.2185
Epoch 21: Validation RMSE = 25.8699
Epoch 22: Validation RMSE = 25.9326
Epoch 23: Validation RMSE = 25.2421
Epoch 24: Validation RMSE = 25.4654
Epoch 25: Validation RMSE = 25.1946
Epoch 26: Validation RMSE = 25.3630
Epoch 27: Validation RMSE = 25.3672
Epoch 28: Validation RMSE = 25.1548
E

In [27]:
model.eval()
all_preds = []

with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        preds = model(X_batch).cpu().numpy()
        all_preds.extend(preds)

test_df["carb"] = all_preds
test_df.to_csv("test_with_predictions.csv", index=False)
